In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
import dataset
import credentials
import sqlite3
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pickle
from Naive_bayes_model import *
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
f = open('nb1.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [3]:
f1 = open('rfp', 'rb')
RFClassifier = pickle.load(f1)
f1.close()

In [4]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [5]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [6]:

def attributeselection(text,query_word):
    attr= []

    #text = "Alleged East Bay serial arsonist arrested #SanFrancisco - http://t.co/ojuHfkHVb2"
    text = nltk.word_tokenize(text)
    poslist = nltk.pos_tag(text)
#     print(poslist)

    verblist = []
    for i in range(len(poslist)):
        if poslist[i][1].startswith('V'):
            verblist.append(poslist[i])
    
    word_count = len(poslist)
#     print(word_count)

    if len(verblist) == 0:
        l = (query_word,'VB')
        verblist.append(l)
    
#     print(verblist,"\n")
    verb_count = len(verblist)
#     print(verb_count)

   # query_word = 'arson'

    words_before = words_after = 0
    for i in range(len(poslist)):
        if poslist[i][0].startswith(query_word):
            words_before = i

    words_after = word_count - words_before - 1

#     print(words_before)
#     print(words_after)
    verbs = []    
    attr.append(word_count)
#     for i in verblist:
#         print(i[0])
#         verbs.append(i[0])
#     attr.append(verbs)
    attr.append(verb_count)
    attr.append(words_before + 1)
#     attr_query_word.append(query_word)
    attr.append(words_before)
    attr.append(words_after)
    
    
        
    column_names = ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
    print(attr,column_names)
    df = pd.DataFrame([attr], columns=column_names)
    arr_x=df.to_numpy(dtype=object)
    print(arr_x,arr_x.shape)
    y_pred = RFClassifier.predict(arr_x)
    print("PRIORITY: ",y_pred)
    
    

In [7]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener,tweet_mode='extended')
        
        stream.filter(languages=["en"],track=['coronavirus'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']
        
        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        if info == 'Relevant' or info == 1 or info == '1':
            attributeselection(text,'coronavirus')
            
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [8]:
if __name__ == '__main__':
    
    hash_tag_list = ['coronavirus']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: RT @Sam1Fleming: Germany’s coronavirus anomaly: high infection rates but few deaths https://t.co/CL0tLEsQOS via @financialtimes

['rt', 'germany', '’', 'coronavirus', 'anomaly', 'high', 'infection', 'rates', 'deaths', 'via']
[22, 1, 8, 7, 14] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[22 1 8 7 14]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @real_defender: AOC and Julian Castro are calling for investigations and resignations of  Republican Senators Burr and Loeffer for unloa…

['rt', 'aoc', 'julian', 'castro', 'calling', 'investigations', 'resignations', 'republican', 'senators', 'burr', 'loeffer', 'unloa…']
[22, 2, 1, 0, 21] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[22 2 1 0 21]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @thompsonstrain1: Come on people...

['rt', 'come', 'people', '...']
[8, 1, 1, 0, 7] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[8 1 1 0 7]] (1, 5)

Tweet: RT @A7_MVT: Damn I gotta warn my mum

['rt', 'damn', 'got', 'ta', 'warn', 'mum']

 Class: 0

Tweet: RT @TIinExile: The last time a Pope asked God to stop a major epidemic was in 1348.

Pope Clement VI  had asked God to stop Black Death pla…

['rt', 'last', 'time', 'pope', 'asked', 'god', 'stop', 'major', 'epidemic', '1348.', 'pope', 'clement', 'vi', 'asked', 'god', 'stop', 'black', 'death', 'pla…']

 Class: 0

Tweet: RT @spectatorindex: Most coronavirus deaths

Italy: 3405
China: 3245
Iran: 1284
Spain: 830
France: 372
US: 171
UK: 144
South Korea: 91
Neth…

['rt', 'coronavirus', 'deaths', 'italy', '3405', 'china', '3245', 'iran', '1284', 'spain', '830', 'france', '372', 'us', '171', 'uk', '144', 'south', 'korea', '91', 'neth…']
[33, 1, 6, 5, 27] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[33 1 6 5 27]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @atonedment: the world: there are thousands of people sick with coronavirus around the world. people

Tweet: RT @thelindsayellis: The WHO has mandated since 2015 that no diseases be named after places.

not that these ghouls give a shit, but just a…

['rt', 'mandated', 'since', '2015', 'diseases', 'named', 'places', 'ghouls', 'give', 'shit', 'a…']
[29, 6, 1, 0, 28] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[29 6 1 0 28]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @d_annyc: He can finally be home, sweatpants, hair tied, chilling with no make up on

['rt', 'finally', 'home', 'sweatpants', 'hair', 'tied', 'chilling', 'make']

 Class: 0

Tweet: RT @chaotictwitch: let them worship in jail please

paul and silas did not have two heads. https://t.co/BUGYaMKl0u

['rt', 'let', 'worship', 'jail', 'please', 'paul', 'silas', 'two', 'heads']

 Class: 0



KeyError: 'created_at'